In [1]:
import os
#from test_helper import Test

# Change to the location of data files
# use ml-latest-small first; then on ml-latest if everything works well
dbfs_dir = 's3://geomapping/outputfolder'
devicestatus_filename = dbfs_dir + '/devicestatus.csv'
movies_filename = dbfs_dir + '/movies.csv'

In [4]:
import pyspark

In [7]:
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")

In [8]:
conf = (pyspark.SparkConf()
        .setAppName('app_name')
        .set('spark.hadoop.fs.s3a.access.key', AWS_ACCESS_KEY_ID)
        .set('spark.hadoop.fs.s3a.secret.key', AWS_SECRET_ACCESS_KEY)
)

In [9]:
sc = pyspark.SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/02 17:22:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
import pandas as pd
devicestatus_df = pd.read_csv(
    f"s3a://geomapping/outputfolder/devicestatus.csv",
    storage_options={
        "key": AWS_ACCESS_KEY_ID,
        "secret": AWS_SECRET_ACCESS_KEY,
        "token": AWS_SESSION_TOKEN,
    }
)


In [27]:
print(devicestatus_df.columns)

Index(['Unnamed: 0', 'Longitude', 'Latitude', 'date', 'Manufacturer',
       'Model_Name', 'deviceid'],
      dtype='object')


In [29]:
alldata=devicestatus_df.drop(columns=['Unnamed: 0'])

In [99]:
alldata.head()
import numpy as np

In [103]:
alldata.replace(to_replace=[None], value=np.nan, inplace=True)
print(alldata.isnull().sum())

Longitude       0
Latitude        0
date            0
Manufacturer    0
Model_Name      0
deviceid        0
dtype: int64


In [101]:
from pyspark.sql import SQLContext
sqlcontext = SQLContext(sc)
s_df = sqlcontext.createDataFrame(alldata)

In [104]:
convergeDist = float(0.1)
iterationDist = float("inf")
distanceMeasure= "euclidean"

In [39]:
rdd=s_df.rdd

In [105]:
from point_utils import *

In [52]:
latlon = rdd.map(lambda x: [LatLonPoint(x[0], x[1]), x[2:]]).cache()


In [59]:
centroids = latlon.map(lambda x: x[0]).takeSample(False, k, 1)
centroids = dict(zip(range(0, k), centroids))


22/05/02 17:51:31 WARN TaskSetManager: Stage 12 contains a task of very large size (1382 KiB). The maximum recommended task size is 1000 KiB.
22/05/02 17:51:31 WARN TaskSetManager: Stage 13 contains a task of very large size (1382 KiB). The maximum recommended task size is 1000 KiB.


In [106]:
points =rdd.map(lambda line: [latlon(line[0], line[1]), line[2:]]).cache()

   

In [107]:
 # initialize random sample centroids
centroids = points.map(lambda point: point[0]).takeSample(False, k)
centroids = dict(zip(range(0,k), centroids))



22/05/02 18:30:50 WARN TaskSetManager: Stage 82 contains a task of very large size (1382 KiB). The maximum recommended task size is 1000 KiB.
22/05/02 18:30:51 WARN TaskSetManager: Stage 83 contains a task of very large size (1382 KiB). The maximum recommended task size is 1000 KiB.


In [108]:
from fix_latlon import *

In [118]:
convergeDist = 1
sumDist = 2
measure ='euclidean'
kPoints=points.map(lambda point: point[0]).takeSample(False, k,1)

22/05/02 18:38:53 WARN TaskSetManager: Stage 87 contains a task of very large size (1382 KiB). The maximum recommended task size is 1000 KiB.
22/05/02 18:38:54 WARN TaskSetManager: Stage 88 contains a task of very large size (1382 KiB). The maximum recommended task size is 1000 KiB.


In [120]:
tempDist = 1.0
while tempDist > convergeDist:
        closest = points.map(
            lambda p: (closestPoint(p, kPoints), (p, 1)))
        pointStats = closest.reduceByKey(
            lambda p1_c1, p2_c2: (p1_c1[0] + p2_c2[0], p1_c1[1] + p2_c2[1]))
        newPoints = pointStats.map(
            lambda st: (st[0], st[1][0] / st[1][1])).collect()

        tempDist = sum(np.sum((kPoints[iK] - p) ** 2) for (iK, p) in newPoints)

        for (iK, p) in newPoints:
            kPoints[iK] = p
        centroids = newPoints.collectAsMap()
print("Final centers: " + str(kPoints))

Final centers: [<fix_latlon.latlon object at 0x7faf95e89b50>, <fix_latlon.latlon object at 0x7faf95e89390>, <fix_latlon.latlon object at 0x7faf95e89a10>, <fix_latlon.latlon object at 0x7faf95e895d0>, <fix_latlon.latlon object at 0x7faf95e89050>]


In [128]:
result_b = []
for idx, centroid in centroids.items():
        print (str(idx) + "," + str(centroid.lat) + "," + str(centroid.lon))
        result_b.append(str(idx) + "," + str(centroid.lat) + "," + str(centroid.lon)) 


0,23416.0,-118.229233586
1,6568.0,-122.085879242
2,11867.0,-117.154222042
3,32214.0,-118.339530341
4,41676.0,-114.611944456


In [136]:
result_b
df = pd.DataFrame(result_b)
df[['index','Latitude','Longitude']]=df[0].str.split(',', expand=True)

In [144]:
df=df.drop(columns=[0])

In [145]:
df.head()

,index,Latitude,Longitude
0,0,23416.0,-118.229233586
1,1,6568.0,-122.085879242
2,2,11867.0,-117.154222042
3,3,32214.0,-118.339530341
4,4,41676.0,-114.611944456


In [ ]:
df.to_csv("next.txt")

install s3fs